In [1]:
import pandas as pd 
import numpy as np
!pip install xlrd

     |████████████████████████████████| 112kB 18.2MB/s eta 0:00:01


In [2]:
df = pd.read_excel('Delhi_Neighborhood.xlsx')
df.head()

,Postalcode,Neighborhood,latitude,longitude
0,110006,Chandni Chowk,28.6505,77.2303
1,110006,Chawri Bazaar,28.6482,77.2270
2,110007,Kamla Nagar,28.6809,77.2046
3,110054,Kashmiri Gate,28.6665,77.2333
4,110005,Karol Bagh,28.6550,77.1888


In [3]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    nu

## Lets get the latitude and longitude of Delhi to draw a map first.

In [4]:
address = 'Delhi, India'

geolocator = Nominatim(user_agent="Delhi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.6517178, 77.2219388.


## Create a map of Delhi with neighborhoods superimposed on top.

In [10]:
map_Delhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalcode, neighborhood in zip(df['latitude'], df['longitude'], df['Postalcode'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Delhi)  
    
map_Delhi

## Use Define Foursquare Credentials and Version

In [11]:

# define Foursquare Credentials and Version
CLIENT_ID = 'HUDBNWSLCX5E2WPPJCPHWJC2Z5QXEGGUPLW44DQFAI3XNHGI' # your Foursquare ID
CLIENT_SECRET = 'ZUEL0EWB1F50JH5LMWMTATNKHKBOBLR0L1IZSHERK5MPDEHU' # your Foursquare Secret
VERSION = '20191206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HUDBNWSLCX5E2WPPJCPHWJC2Z5QXEGGUPLW44DQFAI3XNHGI
CLIENT_SECRET:ZUEL0EWB1F50JH5LMWMTATNKHKBOBLR0L1IZSHERK5MPDEHU


## Let's explore the first neighborhood in our dataframe

In [12]:
df.loc[0, 'Neighborhood']

'Chandni Chowk'